In [1]:
%load_ext dotenv
%dotenv
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner


In [2]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:90.0) Gecko/20100101 Firefox/90.0'
}

def parse_html(content) -> str:
    soup = BeautifulSoup(content, 'html.parser')
    text_content_with_links = soup.get_text()
    return text_content_with_links

def fetch_web_page(url: str) -> str:
    response = requests.get(url, headers=HEADERS)
    return parse_html(response.content)

In [3]:
ddg_search = DuckDuckGoSearchResults()

web_fetch_tool = Tool.from_function(
    func=fetch_web_page,
    name="WebFetcher",
    description="Fetches the content of a web page"
)

prompt_template = "Summarize the following content: {content}"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

summarize_tool = Tool.from_function(
    func=llm_chain.run,
    name="Summarizer",
    description="Summarizes a web page"
)

tools = [ddg_search, web_fetch_tool, summarize_tool]

agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True
)



In [4]:
prompt = "Research how to use the requests library in Python. Use your tools to search and summarize content into a guide on how to use the requests library."

print(agent.run(prompt))



> Entering new AgentExecutor chain...
I should start by searching for information on how to use the requests library in Python. I can use the DuckDuckGo Results JSON tool to search for relevant articles and tutorials. Once I find a useful resource, I can use the WebFetcher to retrieve the content of the page and then use the Summarizer to summarize the information into a guide.

Action: DuckDuckGo Results JSON
Action Input: "how to use requests library in Python"
Observation: [snippet: Cheatsheet. Games. Loops. Python Courses. Python 3 For Beginners. You are here: Home. /Requests. /Using the Requests Library in Python. Using the Requests Library in Python. Author: PFB Staff Writer. Last Updated: August 28, 2020. First things first, let's introduce you to Requests. What is the Requests Resource?, title: Using the Requests Library in Python - PythonForBeginners.com, link: https://www.pythonforbeginners.com/requests/using-requests-in-python], [snippet: Prerequisites. To complete this tu

In [5]:
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [7]:
result = agent.run("Research how to use the requests library in Python. Use your tools to search and summarize content into a guide on how to use the requests library.")
print(result)



> Entering new PlanAndExecute chain...
steps=[Step(value='Begin by searching for relevant documentation and tutorials on how to use the requests library in Python.'), Step(value='Summarize the key features and functionalities of the requests library.'), Step(value='Provide examples of common use cases and code snippets demonstrating how to use the library.'), Step(value='Include any additional tips or best practices for working with the requests library.'), Step(value='Organize the information gathered into a well-structured guide.'), Step(value='Proofread and edit the guide for clarity and accuracy.'), Step(value='Format the guide in a presentable manner.'), Step(value='Add proper citations and references to the original sources of information used.'), Step(value='Review the guide to ensure it covers all important aspects of using the requests library.'), Step(value="Given the above steps taken, respond to the user's original question. \n\n")]

> Entering new AgentExecutor chain...
